# Demo the Liquidator Agent Logic

This notebook:

1. Fetches the latest crvusd contract data into Python objects using `crvusdsim`.
2. Fetches prices and `ExternalMarket`s from the `baseline` scenario.
3. Instantiates a liquidator agent.
4. Checks if there are users to liquidate in the `Controller` and liquidates them.

In [7]:
from crvusdsim.pool import get
from src.agents.liquidator import Liquidator
from src.sim.scenario import Scenario

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
liquidator = Liquidator()

ETH_POOL = "weth"
(
    pool,
    controller,
    collateral_token,
    stablecoin,
    aggregator,
    stableswap_pools,
    peg_keepers,
    policy,
    factory,
) = get(ETH_POOL, bands_data="controller")

In [9]:
config = "../src/configs/scenarios/baseline.json"
scenario = Scenario(config)
markets = scenario.generate_markets()
prices = scenario.generate_pricepaths(
    "../src/configs/prices/1h_1694894242_1700078242.json"
)

[INFO][23:30:07][root]-478786: Reading price config from ../src/configs/scenarios/baseline.json.
[INFO][23:30:07][root]-478786: Fetching 1inch quotes.
[INFO][23:30:13][root]-478786: We have 408960 quotes.
[INFO][23:30:13][root]-478786: Fitting external markets against 1inch quotes.
[INFO][23:30:13][root]-478786: Reading price config from ../src/configs/prices/1h_1694894242_1700078242.json.


In [10]:
# Set External Market Prices
sample = prices[-1].prices
for in_token in markets:
    for out_token in markets[in_token]:
        markets[in_token][out_token].update_price(sample[in_token][out_token])

In [11]:
liquidator.set_paths(controller, stableswap_pools, markets)
# Artificially raise price if necessary to make liquidation profitable
liquidator.paths[0].collat_pool.update_price(5000)

In [12]:
profits, underwater_debt = liquidator.perform_liquidations(controller)

[INFO][23:30:13][root]-478786: There are 7 users to liquidate.
[INFO][23:30:13][root]-478786: Liquidating user 0x99418a168b3eafaf03cc1d00f7c3dbc05e892fd5 with expected profit: [64804.96948549].
[INFO][23:30:13][root]-478786: Executing trade Swap(pool=<SimCurveStableSwapPool address=0x4dece678ceceb27446b35c672dc7d61f30bad69e chain=mainnet>, i=0, j=1, amt=76872321881).
[INFO][23:30:13][root]-478786: Executing trade Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7efdb709ea80>, position=<crvusdsim.pool.crvusd.controller.Position object at 0x7efdb94f9410>, amt=77083671298305832255488, frac=1000000000000000000, i=0, j=1).
[INFO][23:30:13][root]-478786: Executing trade Swap(pool=External Market: WETH -> USDC, i=0, j=1, amt=28344604467166208000).
[INFO][23:30:13][root]-478786: Liquidated user 0x99418a168b3eafaf03cc1d00f7c3dbc05e892fd5 with profit: [64804.96948549].
[INFO][23:30:13][root]-478786: Liquidating user 0x8dbafd8a8e0ef8afdfd6b2b8e2bbfb51d